# Intructions

To run this report, you will first need the following reports

* The latest Aula report can be found [here](https://livecoventryac.sharepoint.com/:f:/t/PlatformTeam/EkSA0F6tMO5PiJeKRIRkzE0Btw5Rr7bQKZMm9uUPFX3mEA?e=XgTpsw)
or [here](https://livecoventryac.sharepoint.com/:f:/t/PlatformTeam/EkJXF1VGYi9OnHshquJOYD0BPdj3TeujYX5RVTbXDH8PYA?e=jKMfWB)

* Enhanced learner report - Use the first tabin the latest SEC report, which is called FL Learner Report, to create a new csv file named Learner_Report.csv
[SEC report can be found here](https://livecoventryac.sharepoint.com/:f:/t/PlatformTeam/EkSA0F6tMO5PiJeKRIRkzE0Btw5Rr7bQKZMm9uUPFX3mEA?e=XgTpsw)
or 
 [here](https://livecoventryac.sharepoint.com/:f:/t/PlatformTeam/EkJXF1VGYi9OnHshquJOYD0BPdj3TeujYX5RVTbXDH8PYA?e=jKMfWB)
 
Both files must be placed in the PT Reporting/Jupyter Notebooks(Python) [Reporting/SLT Report/Data folder](https://livecoventryac.sharepoint.com/:f:/t/PlatformTeam/EqbDy_IXthdIkSyoL8F_TjUB8N1GaQtTJP7D8btD-UN5uw?e=CdqcbD)

The import link for the Aula report can be found here, and must be updated to match your file - [Import Aula report](#aula-report)    

The import link for the Enhanced learner report can be found here, and must be updated to match your file - [Import FL Report](#FL_report)  


Module report - The filter for weekly reporting can be amended here and must be changed weekly - [Filter last step visited date](#cutoff_past_week) 

Degree Report - The filter for weekly reporting can be amended here and must be changed weekly - [Filter last step visited date](#cutoff_past_week_degree) 


Once everything is set up and you are ready to run the report, save the notebook and the click on the 'kernel' tab on the menu bar and select 'Resart & run all'. The excel report will be availibale to download at the end of the notebook.

# Install libraries

In [ ]:
pip install xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
import xlsxwriter
from datetime import datetime, date, timedelta
from IPython.display import display, FileLink

# Import Aula report
<a id="aula_report"></a>

In [ ]:
# Amend file path below to location of aula report
aula_report = pd.read_csv('./Data/Aula_Report.csv', dtype={'STUDENT_ID':'str'})
aula_report.rename(columns={'STUDENT_ID':'learner_identifier'},inplace=True)
aula_report

# Import FL Learner report csv
<a id="FL_report"></a>

In [ ]:
# Amend file path below to location of FL learner report
learner_report = pd.read_csv('./Data/Learner_Report.csv', encoding='latin-1',  dtype={'run_code':'str','course_code':'str'})  # changes datatype on run code and course code to string
                    
learner_report

# Module Report

### Create one per program sheet

In [ ]:
one_per_program = learner_report.copy()
# deduplicate by learner_identifier and run code
one_per_program = one_per_program.drop_duplicates(subset=['learner_identifier', 'run_code'])
one_per_program

### Create list of learners who accessed FL during the past week
<a id="cutoff_past_week"></a>

In [ ]:
last_week_program = learner_report.copy()
# Edit dates to create last week range
last_week_program = last_week_program[(last_week_program['date_of_last_step_visit'] >= '2023-06-19')
                                      & (last_week_program['date_of_last_step_visit'] <= '2023-06-25')]
last_week_program = last_week_program.drop_duplicates(subset=['learner_identifier', 'run_code'])
last_week_program


### Create step completion sheet

In [ ]:
step_completion = learner_report.copy()
# step_completion = step_completion['run_start_date']

# Convert the date column to datetime type if it's not already in that format
step_completion['run_start_date'] = pd.to_datetime(step_completion['run_start_date'])

# Calculate the date 7 days ago from the current date
current_date = datetime.now().date()
past_date = current_date - pd.DateOffset(days=7)

# Filter the DataFrame to exclude the past 7 days
step_completion = step_completion[step_completion['run_start_date'] < past_date]
step_completion

In [ ]:
# add in %
step_completion['steps_completed'] = step_completion['steps_completed'].str.replace('%', '')
step_completion['steps_completed'] = step_completion['steps_completed'].astype(int)

step_completion

### Create Step completion pivot table

In [ ]:
step_completion_pivot = pd.pivot_table(step_completion, values='steps_completed', index=['run_code'], aggfunc='mean',  margins=True, margins_name='Total')
step_completion_pivot['steps_completed'] = step_completion_pivot['steps_completed']/100
step_completion_pivot = step_completion_pivot.reset_index()
step_completion_pivot

# Degree Report

### Create list of invites/LTI links sent

In [ ]:
invites_degree = aula_report.copy()
invites_degree = invites_degree.drop_duplicates(subset=['learner_identifier', 'DEGREE_COURSE'])
invites_degree

### Create one learner per degree sheet

In [ ]:
one_per_degree = learner_report.copy()
# Deduplicate by learner_id and run code
one_per_degree = one_per_degree.drop_duplicates(subset=['learner_identifier', 'Degree'])
one_per_degree


### Create list of learners who accessed FL during the past week by degree
<a id="cutoff_past_week_degree"></a>

In [ ]:
last_week_degree = learner_report.copy()
last_week_degree = last_week_degree[(last_week_degree['date_of_last_step_visit'] >= '2023-06-19') & (last_week_degree['date_of_last_step_visit'] <= '2023-06-25')]
#remove duplicates by learner_id and run_code
last_week_degree = last_week_degree.drop_duplicates(subset=['learner_identifier', 'Degree'])
last_week_degree



# Output to Excel

In [ ]:
file_path = 'SLT_Report.xlsx'

# Save the DataFrames to the Excel workbook
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    learner_report.to_excel(writer, sheet_name='Enhanced Learner Report', index=False)
    aula_report.to_excel(writer, sheet_name='Aula report', index=False)
    one_per_program.to_excel(writer, sheet_name='1 per program', index=False)
    last_week_program.to_excel(writer, sheet_name='Last week program', index=False)
    step_completion_pivot.to_excel(writer, sheet_name='Step completion', index=False)
    invites_degree.to_excel(writer, sheet_name='Invites Degree', index=False)
    one_per_degree.to_excel(writer, sheet_name='1 per Degree', index=False)
    last_week_degree.to_excel(writer, sheet_name='Last week degree', index=False)

# Display the file path as a link
display(FileLink(file_path))